In [1]:
using Unitful
using CairoMakie
using JLD2
using Interpolations
using NumericalIntegration
import SpecialFunctions: gamma, erfcx
import PhysicalConstants.CODATA2018: h, k_B, R_∞, c_0, m_e, m_u, e, ε_0, a_0

In [2]:
const Ar_H = 1.007975  # Atomic weight of hydrogen
const hc_k = h * c_0 / k_B
const hminusχ = 0.754u"eV"
const saha_const = h^2 / (2 * π * m_e * k_B)  # Constant for Saha equation
const σ_thomson = (e^4 / (6π * ε_0^2 * m_e^2 * c_0^4)) |> u"m^2"  
const αl_const = (e^2 / (4 * ε_0 * m_e * c_0^2)) |> u"m"   # constant for line extinction

@derived_dimension NumberDensity Unitful.𝐋^-3
@derived_dimension PerLength Unitful.𝐋^-1

## Recipes for continuum extinction

In [3]:
#=----------------------------------------------------------------------------
        Recipes from Wishart (1979) and Broad and Reinhardt (1976)
----------------------------------------------------------------------------=#
const wbr_λ = [     18, 19.6, 21.4, 23.6, 26.4, 29.8, 34.3, 40.4, 49.1, 62.6,  121, 139,
                   164,  175,  200,  225,  250,  275,  300,  325,  350,  375,  400, 425,
                   450,  475,  500,  525,  550,  575,  600,  625,  650,  675,  700, 725,
                   750,  775,  800,  825,  850,  875,  900,  925,  950,  975, 1000, 1025,
                  1050, 1075, 1100, 1125, 1150, 1175, 1200, 1225, 1250, 1275, 1300, 1325,
                  1350, 1375, 1400, 1425, 1450, 1475, 1500, 1525, 1550, 1575, 1600, 1610,
                  1620, 1630]   # in nm
const wbr_σ = [0.067, 0.088, 0.117, 0.155, 0.206, 0.283, 0.414, 0.703,  1.24,  2.33,
                5.43,  5.91,  7.29, 7.918, 9.453, 11.08, 12.75, 14.46, 16.19, 17.92,
               19.65, 21.35, 23.02, 24.65, 26.24, 27.77, 29.23, 30.62, 31.94, 33.17,
               34.32, 35.37, 36.32, 37.17, 37.91, 38.54, 39.07, 39.48, 39.77, 39.95,
               40.01, 39.95, 39.77, 39.48, 39.06, 38.53, 37.89, 37.13, 36.25, 35.28,
               34.19, 33.01, 31.72, 30.34, 28.87, 27.33, 25.71, 24.02, 22.26, 20.46,
               18.62, 16.74, 14.85, 12.95, 11.07, 9.211, 7.407, 5.677, 4.052, 2.575,
               1.302, 0.8697, 0.4974, 0.1989]  # in 1e-22 m^2
const wbr_bf_interp = linear_interpolation(wbr_λ, wbr_σ, extrapolation_bc=0)

#=----------------------------------------------------------------------------
                            Recipes from Stilley
----------------------------------------------------------------------------=#
const stilley_ff_λ = [0.0, 303.8, 455.6, 506.3, 569.5, 650.9, 759.4, 911.3,
                      1013.0, 1139.0, 1302.0, 1519.0, 1823.0, 2278.0, 3038.0,
                      4556.0, 9113.0]  # in nm
const stilley_ff_t = 5040.0 ./ [0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2,
                                1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0]  # in K
const stilley_ff_table =
   [[0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00  #=    0.0 nm
=#   0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00];
    [3.44e-02 4.18e-02 4.91e-02 5.65e-02 6.39e-02 7.13e-02 7.87e-02 8.62e-02  #=  303.8 nm
=#   9.36e-02 1.01e-01 1.08e-01 1.16e-01 1.23e-01 1.30e-01 1.38e-01 1.45e-01];
    [7.80e-02 9.41e-02 1.10e-01 1.25e-01 1.40e-01 1.56e-01 1.71e-01 1.86e-01  #=  455.6 nm
=#   2.01e-01 2.16e-01 2.31e-01 2.45e-01 2.60e-01 2.75e-01 2.89e-01 3.03e-01];
    [9.59e-02 1.16e-01 1.35e-01 1.53e-01 1.72e-01 1.90e-01 2.08e-01 2.25e-01  #=  506.3 nm
=#   2.43e-01 2.61e-01 2.78e-01 2.96e-01 3.13e-01 3.30e-01 3.47e-01 3.64e-01];
    [1.21e-01 1.45e-01 1.69e-01 1.92e-01 2.14e-01 2.36e-01 2.58e-01 2.80e-01  #=  569.5 nm
=#   3.01e-01 3.22e-01 3.43e-01 3.64e-01 3.85e-01 4.06e-01 4.26e-01 4.46e-01];
    [1.56e-01 1.88e-01 2.18e-01 2.47e-01 2.76e-01 3.03e-01 3.31e-01 3.57e-01  #=  650.9 nm
=#   3.84e-01 4.10e-01 4.36e-01 4.62e-01 4.87e-01 5.12e-01 5.37e-01 5.62e-01];
    [2.10e-01 2.53e-01 2.93e-01 3.32e-01 3.69e-01 4.06e-01 4.41e-01 4.75e-01  #=  759.4 nm
=#   5.09e-01 5.43e-01 5.76e-01 6.08e-01 6.40e-01 6.72e-01 7.03e-01 7.34e-01];
    [2.98e-01 3.59e-01 4.16e-01 4.70e-01 5.22e-01 5.73e-01 6.21e-01 6.68e-01  #=  911.3 nm
=#   7.15e-01 7.60e-01 8.04e-01 8.47e-01 8.90e-01 9.32e-01 9.73e-01 1.01e+00];
    [3.65e-01 4.39e-01 5.09e-01 5.75e-01 6.39e-01 7.00e-01 7.58e-01 8.15e-01  #= 1013.0 nm
=#   8.71e-01 9.25e-01 9.77e-01 1.03e+00 1.08e+00 1.13e+00 1.18e+00 1.23e+00];
    [4.58e-01 5.50e-01 6.37e-01 7.21e-01 8.00e-01 8.76e-01 9.49e-01 1.02e+00  #= 1139.0 nm
=#   1.09e+00 1.15e+00 1.22e+00 1.28e+00 1.34e+00 1.40e+00 1.46e+00 1.52e+00];
    [5.92e-01 7.11e-01 8.24e-01 9.31e-01 1.03e+00 1.13e+00 1.23e+00 1.32e+00  #= 1302.0 nm
=#   1.40e+00 1.49e+00 1.57e+00 1.65e+00 1.73e+00 1.80e+00 1.88e+00 1.95e+00];
    [7.98e-01 9.58e-01 1.11e+00 1.25e+00 1.39e+00 1.52e+00 1.65e+00 1.77e+00  #= 1519.0 nm
=#   1.89e+00 2.00e+00 2.11e+00 2.21e+00 2.32e+00 2.42e+00 2.51e+00 2.61e+00];
    [1.14e+00 1.36e+00 1.58e+00 1.78e+00 1.98e+00 2.17e+00 2.34e+00 2.52e+00  #= 1823.0 nm
=#   2.68e+00 2.84e+00 3.00e+00 3.15e+00 3.29e+00 3.43e+00 3.57e+00 3.70e+00];
    [1.77e+00 2.11e+00 2.44e+00 2.75e+00 3.05e+00 3.34e+00 3.62e+00 3.89e+00  #= 2278.0 nm
=#   4.14e+00 4.39e+00 4.63e+00 4.86e+00 5.08e+00 5.30e+00 5.51e+00 5.71e+00];
    [3.10e+00 3.71e+00 4.29e+00 4.84e+00 5.37e+00 5.87e+00 6.36e+00 6.83e+00  #= 3038.0 nm
=#   7.28e+00 7.72e+00 8.14e+00 8.55e+00 8.95e+00 9.33e+00 9.71e+00 1.01e+01];
    [6.92e+00 8.27e+00 9.56e+00 1.08e+01 1.19e+01 1.31e+01 1.42e+01 1.52e+01  #= 4556.0 nm
=#   1.62e+01 1.72e+01 1.82e+01 1.91e+01 2.00e+01 2.09e+01 2.17e+01 2.25e+01];
    [2.75e+01 3.29e+01 3.80e+01 4.28e+01 4.75e+01 5.19e+01 5.62e+01 6.04e+01  #= 9133.0 nm
=#   6.45e+01 6.84e+01 7.23e+01 7.60e+01 7.97e+01 8.32e+01 8.67e+01 9.01e+01]]
const stilley_ff_interp = linear_interpolation((stilley_ff_λ, stilley_ff_t[end:-1:1]),
                             stilley_ff_table[:, end:-1:1], extrapolation_bc=Line())

"""
    calc_hminus_density(
        h_neutral_density::NumberDensity,
        temperature::Unitful.Temperature,
        electron_density::NumberDensity
    )

Compute H minus populations based on electron density, temperature, and
density of neutral hydrogen atoms.
"""
function calc_hminus_density(
    h_neutral_density::NumberDensity,
    temperature::Unitful.Temperature,
    electron_density::NumberDensity
)
    tmp = (ustrip(saha_const) / ustrip(temperature |> u"K"))^(3/2) * u"m^3"
    ϕ = tmp * exp(hminusχ / (k_B * temperature)) / 4
    return (ϕ * h_neutral_density) * electron_density
end


"""
    α_thomson(electron_density::NumberDensity)

Compute the Thomson extinction as a function of electron density.
"""
α_thomson(electron_density::NumberDensity) = σ_thomson * electron_density


"""
    α_hminus_bf(
        λ::Unitful.Length,
        temperature::Unitful.Temperature,
        h_neutral_density::NumberDensity,
        electron_density::NumberDensity
    )

Compute extinction from H minus ion, from input H minus populations. Uses recipe from
[Wishart (1979)](https://ui.adsabs.harvard.edu/abs/1979MNRAS.187P..59W) for λ down to 175 nm,
and recipe from [Broad and Reinhardt (1976)](https://ui.adsabs.harvard.edu/abs/1976PhRvA..14.2159B)
for λ=164 nm and below, following the recommendation from Mathisen (1984, MSC thesis).
"""
function α_hminus_bf(
    λ::Unitful.Length,
    temperature::Unitful.Temperature,
    h_neutral_density::NumberDensity,
    electron_density::NumberDensity
)
    λi = ustrip(λ |> u"nm")   # convert to units of table
    κ = wbr_bf_interp(λi)::Float64 * 1e-22u"m^2"
    stimulated_emission = exp(-hc_k / (λ * temperature))
    h_minus_density = calc_hminus_density(h_neutral_density, temperature, electron_density)
    return (κ * h_minus_density * (1 - stimulated_emission)) |> u"m^-1"
end


"""
    σ_hminus_ff(
        λ::Unitful.Length,
        temperature::Unitful.Temperature,
        h_neutral_density::NumberDensity,
        electron_density::NumberDensity
    )

Compute free-free cross section coefficient from H minus ion, for a given wavelength
and temperature. Units are m^5, needs to be multiplied by electron density and
density of neutral hydrogen atoms to obtain linear extinction. Interpolates table from
[Stilley & Callaway (1970)](https://ui.adsabs.harvard.edu/abs/1970ApJ...160..245S/abstract),
page 255, which is valid for λ up to 9113 nm.
"""
function α_hminus_ff(
    λ::Unitful.Length,
    temperature::Unitful.Temperature,
    h_neutral_density::NumberDensity,
    electron_density::NumberDensity
)
    λi = ustrip(λ |> u"nm")   # convert to units of table
    temp = ustrip(temperature |> u"K")
    kappa = max(0.0, stilley_ff_interp(λi, temp)::Float64) * 1e-29u"m^4/N"
    σ = k_B * temperature * kappa 
    return (σ * h_neutral_density * electron_density) |> u"m^-1"
end

α_hminus_ff

In [4]:
function α_cont(
    λ::Unitful.Length,
    temperature::Unitful.Temperature,
    h_neutral_density::NumberDensity,
    electron_density::NumberDensity
)
    return (α_hminus_bf(λ, temperature, h_neutral_density, electron_density) + 
            α_hminus_ff(λ, temperature, h_neutral_density, electron_density) +
            α_thomson(electron_density))
end

α_cont (generic function with 1 method)

## New datatypes



In [7]:
struct Atmosphere3D{}
    nx::Int64
    ny::Int64
    nz::Int64
    x
    y
    z
    temperature
    velocity_x
    velocity_y
    velocity_z
    electron_density
    hydrogen1_density  # neutral hydrogen across all levels
    proton_density
end


struct Atmosphere1D{}
    nz::Int64
    z
    temperature
    velocity_z
    electron_density
    hydrogen1_density  # neutral hydrogen across all levels
    proton_density
end


function Base.getindex(a::Atmosphere3D, i, j)
    @assert (i > 0) and (i <= a.ny) "y index out of range"
    @assert (j > 0) and (i <= a.nx) "x index out of range"
    return Atmosphere1D(
        a.nz,
        a.z,
        a.temperature[:, i, j],
        a.velocity_z[:, i, j],
        a.electron_density[:, i, j],
        a.hydrogen1_density[:, i, j],
        a.proton_density[:, i, j],
    )
end

## Other useful functions

In [95]:
i_units = u"kW / (m^2 * nm * sr)"

"""
Planck function for wavelength units.
"""
function blackbody_λ(λ, temp) 
    radiation = 2h * c_0^2 * λ^-5 / (exp(h * c_0 / k_B / (λ * temp)) - 1)
    return radiation |> i_units
end

"""
    incline_atmos(atmos_in::AbstractAtmos3D, μ::Real, φ::Real)

Transforms a 3D atmosphere into an inclined coordinate system, given by a rotation
by a polar angle θ, given by μ = cos(θ) and an azimuthal angle φ. The output is
an atmosphere of the same type, where the quantities have been interpolated to the
same number of depth points and the vector quantities projected onto the new axes.

Assumes the following:

1. Order of the axes in 3D arrays is (z, y, x)
2. Horizontally periodic boundary conditions
3. The first index in the height direction is fixed in the polar rotation
4. A right handed system, so that:

```
      z  ↑.           - θ is the clockwise polar rotation from the z axis
         |  .         - φ is the clockwise azimuthal rotation from the x axis
         |    .
         |      .
         |        *
         |      . .
         | θ  .   .
         |  .     .
         |.       .
         +--------.-----→ y
        /  .      .   /
       /  φ  .    .
      /         . . /
     /_ _ _ _ _ _ .
    /
  x
```
"""
function incline_atmos(atmos_in::Atmosphere3D, μ::Real, φ::Real)
    atmos = deepcopy(atmos_in)
    dx = ustrip(abs(atmos.x[2] - atmos.x[1]))
    dy = ustrip(abs(atmos.y[2] - atmos.y[1]))
    for name in fieldnames(typeof(atmos))
        var = getfield(atmos, name)
        var_in = getfield(atmos_in, name)
        if ndims(var) == 3
            incline_data!(var_in, var, ustrip(atmos.z), dx, dy, μ, φ)
        end
    end
    # adjust physical sizes
    atmos.z ./= μ
    atmos.y .*= sin(φ) + μ * cos(φ)
    atmos.x .*= cos(φ) + μ * sin(φ)
    # project velocity
    project_vector!(atmos.velocity_x, atmos.velocity_y, atmos.velocity_z, μ, φ)
    return atmos
end


"""
    project_vector!(vx::A, vy::A, vz::A, μ::Real, φ::Real)

Projects a vector in 3D space according to the rotation by an polar angle
θ, given by μ = cos(θ) and an azimuthal angle φ. The inputs are the
vector components in the x, y, and z axes, for an array of a given size
(typically 3D).

Assumes the following:

1. θ is the clockwise polar rotation from the z axis
2. φ is the clockwise azimuthal rotation from the x axis
3. The rotation of the vector is the same as the (inverse) rotation from the
   axes (x, y, z) into a new system (x', y', z') given by the rotation matrix:

```
               Polar          Azimuthal
   ⎡x'⎤ = ⎡cosθ  0  -sinθ⎤⎡ cosφ  sinφ  0⎤⎡x⎤
   ⎜y'⎥   ⎜  0   1    0  ⎥⎜-sinφ  cosφ  0⎥⎜y⎥
   ⎣z'⎦   ⎣sinθ  0   cosθ⎦⎣   0     0   1⎦⎣z⎦

   ⌈x'⎤ = ⎡cosθcosφ  cosθsinφ  -sinφ⎤⎡x⎤
   |y'⎥   ⎜ -sinφ      cosφ       0 ⎥⎜y⎥
   ⌊z'⎦   ⎣sinθcosφ  sinθsinφ   cosθ⎦⎣z⎦
```
"""
function project_vector!(vx::A, vy::A, vz::A, μ::Real, φ::Real) where A <: AbstractArray{<:Any}
    cosθ = μ
    sinθ = sqrt(1 - μ^2)
    cosφ = cos(φ)
    sinφ = sin(φ)
    Threads.@threads for i in eachindex(vx)
        proj_x = vx[i]*cosθ*cosφ + vy[i]*cosθ*sinφ - vz[i]*sinθ
        proj_y = -vx[i]*sinφ + vy[i]*cosφ
        proj_z = vx[i]*sinθ*cosφ + vy[i]*sinθ*sinφ + vz[i]*cosθ
        vx[i] = proj_x
        vy[i] = proj_y
        vz[i] = proj_z
    end
    return nothing
end



function incline_data!(
        data_in::AbstractArray{<: Any, 3},
        data_out::AbstractArray{<: Any, 3},
        z::AbstractVector,
        dx::Real,
        dy::Real,
        μ::Real,
        ϕ::Real,
)
    # Invert θ so that inclination goes towards increasing x or y values
    μ = -μ
    @assert abs(μ) > 0 "μ must be non-zero"
    θ = acos(μ)
    sinθ = sin(θ)
    tanθ = sinθ / μ
    cosϕ = cos(ϕ)
    sinϕ = sin(ϕ)
    ∂x∂z = tanθ * cosϕ
    ∂y∂z = tanθ * sinϕ
    nz, ny, nx = size(data_in)
    @assert nz == length(z)
    @assert dx != 0
    @assert dy != 0
    ε = 1.0e-6
    if (abs(∂x∂z) < ε) & (abs(∂y∂z) < ε)
        data_out .= data_in
        return nothing
    elseif (abs(∂x∂z) > ε) & (abs(∂y∂z) > ε)
        dual = true
        buf = similar(data_in)
    else
        dual = false
        buf = data_in
    end

    if abs(∂x∂z) > ε
        Threads.@threads for l in 1:nx
            for n in 1:nz
                shift_x = ∂x∂z * z[n] / (nx*dx)
                k, a, b = _linear_coeffs(shift_x, nx)
                p0, p1 = _linear_stencil(l, k, nx)
                for m in 1:ny
                    data_out[n,m,l] = a*data_in[n,m,p0] + b*data_in[n,m,p1]
                end
            end
        end
    end

    if abs(∂y∂z) > ε
        if dual
            buf .= data_out
        end
        Threads.@threads for m in 1:ny
            for n in 1:nz
                shift_y = ∂y∂z * z[n] / (ny*dy)
                k, a, b = _linear_coeffs(shift_y, ny)
                p0, p1 = _linear_stencil(m, k, ny)
                for l in 1:nx
                    data_out[n,m,l] = a*buf[n,p0,l] + b*buf[n,p1,l]
                end
            end
        end
    end
    return nothing
end


"""
Computes coefficients for linear piecewise interpolation.
"""
function _linear_coeffs(shift, n)
    shift = mod(shift, 1)
    if shift < 0
        shift += 1
    end
    shift *= n
    k = floor(Int64, shift)
    p = shift - k
    k += n
    q = 1 - p
    return (k, q, p)
end


"""
Get stencil for linear piecewise interpolation.
"""
function _linear_stencil(index, k, n)
    p0 = mod(index + k - 1, n) + 1
    p1 = mod(index + k, n) + 1
    return (p0, p1)
end

_linear_stencil

## Hands-on

Please download the [atmosphere file](https://www.uio.no/studier/emner/matnat/astro/AST4310/h25/files/quasi_solar.jld2) to your computer, and place it in the same directory as this notebook.



In [8]:
@load "quasi_solar.jld2" atm

1-element Vector{Symbol}:
 :atm